## EJERCICIO 2

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import os

def exportar_datos(dataframe, formato='csv', nombre_archivo=None, ruta_destino='./exportaciones/', **kwargs):
    """
    Exporta un DataFrame a archivo CSV o Parquet.
    
    Parameters:
    -----------
    dataframe : pandas.DataFrame
        DataFrame con los datos a exportar
    formato : str, default 'csv'
        Formato de exportación: 'csv' o 'parquet'
    nombre_archivo : str, optional
        Nombre del archivo de salida. Si es None, se genera automáticamente
    ruta_destino : str, default './exportaciones/'
        Ruta donde se guardará el archivo
    **kwargs : dict
        Argumentos adicionales para pandas.to_csv() o pyarrow.parquet.write_table()
        
    Returns:
    --------
    str
        Ruta completa del archivo exportado
        
    Raises:
    -------
    ValueError
        Si el formato no es 'csv' o 'parquet'
    Exception
        Si ocurre algún error durante la exportación
    """
    
    # Validar formato
    formato = formato.lower()
    if formato not in ['csv', 'parquet']:
        raise ValueError(f"Formato '{formato}' no soportado. Use 'csv' o 'parquet'")
    
    # Validar que el DataFrame no esté vacíoo
    if dataframe is None or dataframe.empty:
        raise ValueError("El DataFrame está vacío o es None")
    
    # Crear directorio de destino si no existe
    os.makedirs(ruta_destino, exist_ok=True)
    
    # Generar nombre de archivo si no se proporciona
    if nombre_archivo is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_archivo = f"datos_exportados_{timestamp}.{formato}"
    else:
        # Asegurar que tenga la extensión correcta
        # Primero removemos cualquier extensión existente
        if '.' in nombre_archivo:
            nombre_base = nombre_archivo.rsplit('.', 1)[0]  # Divide por el último punto
            nombre_archivo = f"{nombre_base}.{formato}"
        else:
            nombre_archivo = f"{nombre_archivo}.{formato}"
    
    # Ruta completa del archivo
    ruta_completa = os.path.join(ruta_destino, nombre_archivo)
    
    try:
        if formato == 'csv':
            # Argumentos por defecto para CSV
            csv_args = {
                'index': False,
                'encoding': 'utf-8',
                'sep': ','
            }
            # Actualizar con argumentos proporcionados
            csv_args.update(kwargs)
            
            # Exportar a CSV
            dataframe.to_csv(ruta_completa, **csv_args)
            print(f"Datos exportados exitosamente a CSV: {ruta_completa}")
            
        elif formato == 'parquet':
            # Convertir DataFrame a tabla de PyArrow
            tabla = pa.Table.from_pandas(dataframe)
            
            # Exportar a Parquet
            pq.write_table(tabla, ruta_completa, **kwargs)
            print(f"Datos exportados exitosamente a Parquet: {ruta_completa}")
        
        return ruta_completa
        
    except Exception as e:
        # Limpiar archivo si la exportación falla
        if os.path.exists(ruta_completa):
            os.remove(ruta_completa)
        raise Exception(f"Error al exportar datos: {str(e)}")

# Función auxiliar para exportar múltiples formatos
def exportar_multiple_formatos(dataframe, nombre_base, ruta_destino='./exportaciones/', formatos=['csv', 'parquet'], **kwargs):
    """
    Exporta un DataFrame a múltiples formatos simultáneamente.
    
    Parameters:
    -----------
    dataframe : pandas.DataFrame
        DataFrame con los datos a exportar
    nombre_base : str
        Nombre base para los archivos (sin extensión)
    ruta_destino : str
        Ruta donde se guardarán los archivos
    formatos : list
        Lista de formatos a exportar: ['csv', 'parquet']
    **kwargs : dict
        Argumentos adicionales para las funciones de exportación
        
    Returns:
    --------
    dict
        Diccionario con las rutas de los archivos exportados por formato
    """
    
    rutas_exportadas = {}
    
    for formato in formatos:
        try:
            nombre_archivo = f"{nombre_base}.{formato}"
            ruta = exportar_datos(
                dataframe=dataframe,
                formato=formato,
                nombre_archivo=nombre_archivo,
                ruta_destino=ruta_destino,
                **kwargs
            )
            rutas_exportadas[formato] = ruta
        except Exception as e:
            print(f"Error al exportar {formato.upper()}: {str(e)}")
            rutas_exportadas[formato] = None
    
    return rutas_exportadas

# Ejemplo de uso y pruebas
if __name__ == "__main__":
    # Crear datos de ejemplo para pruebas
    datos_ejemplo = pd.DataFrame({
        'id': [1, 2, 3, 4, 5],
        'nombre': ['Ana', 'Luis', 'Maria', 'Carlos', 'Elena'],
        'departamento': ['Ventas', 'IT', 'RH', 'Ventas', 'IT'],
        'salario': [50000, 75000, 60000, 55000, 80000],
        'fecha_contratacion': pd.date_range('2020-01-01', periods=5, freq='M')
    })
    
    print("=== PRUEBAS DE EXPORTACIÓN ===")
    
    # Prueba 1: Exportar a CSV
    try:
        ruta_csv = exportar_datos(datos_ejemplo, formato='csv', nombre_archivo='empleados_ejemplo')
        print(f"✓ CSV exportado: {ruta_csv}")
    except Exception as e:
        print(f"✗ Error CSV: {e}")
    
    # Prueba 2: Exportar a Parquet
    try:
        ruta_parquet = exportar_datos(datos_ejemplo, formato='parquet', nombre_archivo='empleados_ejemplo')
        print(f"✓ Parquet exportado: {ruta_parquet}")
    except Exception as e:
        print(f"✗ Error Parquet: {e}")
    
    # Prueba 3: Exportar a múltiples formatos
    try:
        rutas = exportar_multiple_formatos(
            datos_ejemplo, 
            nombre_base='empleados_completo',
            formatos=['csv', 'parquet']
        )
        print(f"✓ Múltiples formatos exportados: {rutas}")
    except Exception as e:
        print(f"✗ Error múltiples formatos: {e}")
    
    # Prueba 4: Verificar archivos exportados
    print("\n=== VERIFICACIÓN DE ARCHIVOS ===")
    export_dir = './exportaciones/'
    if os.path.exists(export_dir):
        archivos = os.listdir(export_dir)
        for archivo in archivos:
            tamaño = os.path.getsize(os.path.join(export_dir, archivo))
            print(f"📁 {archivo} - {tamaño} bytes")

=== PRUEBAS DE EXPORTACIÓN ===
Datos exportados exitosamente a CSV: ./exportaciones/empleados_ejemplo.csv
✓ CSV exportado: ./exportaciones/empleados_ejemplo.csv
Datos exportados exitosamente a Parquet: ./exportaciones/empleados_ejemplo.parquet
✓ Parquet exportado: ./exportaciones/empleados_ejemplo.parquet
Datos exportados exitosamente a CSV: ./exportaciones/empleados_completo.csv
Datos exportados exitosamente a Parquet: ./exportaciones/empleados_completo.parquet
✓ Múltiples formatos exportados: {'csv': './exportaciones/empleados_completo.csv', 'parquet': './exportaciones/empleados_completo.parquet'}

=== VERIFICACIÓN DE ARCHIVOS ===
📁 empleados_completo.csv - 202 bytes
📁 empleados_completo.parquet - 3955 bytes
📁 empleados_ejemplo.csv - 202 bytes
📁 empleados_ejemplo.parquet - 3955 bytes


C:\Users\adrian.giron\AppData\Local\Temp\ipykernel_22708\1395587885.py:147: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'fecha_contratacion': pd.date_range('2020-01-01', periods=5, freq='M')
